In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.baseline_functions as base

# restore saved variables
%store -r summary_drug_two_year

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test


In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
data = data.drop(['fta_risk_score_raw','nca_risk_score_raw','pvf_risk_score_raw', 
                  'fta_calc', 'nca_calc', 'pvf_calc'], axis=1)
x = data.loc[:,:'current_violence']
y = data['recid_M_six_month'].values

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(X=x,
                                 Y=y,
                                 C=c,
                                 seed=816)
#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(X=x,
                           Y=y,
                           C=c,
                           seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(X=x,
                             Y=y, 
                             C=c,
                             seed=816)

#### CART
depth = [7,8,9]
cart_summary = base.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### Random Forest
n_estimators =  [40,50]
depth = [6,7]
rf_summary = base.RF(X=x,
                     Y=y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.01]
depth = [6]
n_estimators = [200]
xgb_summary = base.XGB(X=x,
                       Y=y, 
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators, 
                       seed=816)

#### save results
summary_drug_two_year = {"Logistic": logistic_summary,
                         "Lasso": lasso_summary,
                         "LinearSVM": svm_summary,
                         "CART": cart_summary,
                         "RF": rf_summary,
                         "XGBoost": xgb_summary}
%store summary_drug_two_year

model_id, score_thresholds 0 {'rank_abs': [7096]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7393]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7374]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7357]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7229]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7114]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7385]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7370]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7359]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [7231]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [127]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [146]}
get_disparity_predefined_group()
model_

In [4]:
results = [[model_name, np.mean(model_summary["holdout_test_auc"]), np.mean(model_summary["auc_diffs"])] \
           for model_name, model_summary in summary_drug_two_year.items()]
results 

[['Logistic', 0.7362210832838358, 0.002154589736028445],
 ['Lasso', 0.7361711569899864, 0.0022790986266323587],
 ['LinearSVM', 0.7411897060583122, 0.0018461359819135215],
 ['CART', 0.74540271088215, 0.019285906372375927],
 ['RF', 0.7440400435425856, 0.016497322084035804],
 ['XGBoost', 0.7516974917151247, 0.016649510859739626]]

In [5]:
params = [[model_name, model_summary['best_param']] for model_name, model_summary in summary_drug_two_year.items()]
params

[['Logistic', [{'C': 0.01}, {'C': 0.01}, {'C': 0.1}, {'C': 0.1}, {'C': 0.01}]],
 ['Lasso', [{'C': 1}, {'C': 0.1}, {'C': 1}, {'C': 1}, {'C': 0.1}]],
 ['LinearSVM', [{'C': 1}, {'C': 0.1}, {'C': 0.1}, {'C': 0.1}, {'C': 0.1}]],
 ['CART',
  [{'max_depth': 7},
   {'max_depth': 7},
   {'max_depth': 7},
   {'max_depth': 7},
   {'max_depth': 7}]],
 ['RF',
  [{'max_depth': 7, 'n_estimators': 50},
   {'max_depth': 7, 'n_estimators': 50},
   {'max_depth': 7, 'n_estimators': 50},
   {'max_depth': 7, 'n_estimators': 50},
   {'max_depth': 7, 'n_estimators': 50}]],
 ['XGBoost',
  [{'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200},
   {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200},
   {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200},
   {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200},
   {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200}]]]

In [6]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\Baselines\\Six Month\\"
results = [["Misdemeanor", np.str((round(np.mean(logistic_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(logistic_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(lasso_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(lasso_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(svm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(svm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(cart_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(rf_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(rf_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(xgb_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(xgb_summary['holdout_test_auc']), 3)) + ")"]]
with open(path + 'Six Month Baseline Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)